In [1]:
#!/usr/bin/env python

In [2]:
import os
import sys
if os.path.exists('/home/chieh/code/wPlotLib'):
	sys.path.insert(0,'/home/chieh/code/wPlotLib')
if os.path.exists('/home/chieh/code/wuML'):
	sys.path.insert(0,'/home/chieh/code/wuML')

In [3]:
import wuml
import numpy as np
import torch
import wplotlib
from wuml import HSIC

In [4]:
data = wuml.wData(xpath='../../data/wine.csv', ypath='../../data/wine_label.csv', randomly_shuffle_batch=True,
					batch_size=20, label_type='discrete', encode_discrete_label_to_one_hot=True )

Remember that you are trying to "minimize" this objective

In [5]:
def costFunction(x, y, ŷ, ind):
	H = HSIC(ŷ, y, X_kernel='linear', Y_kernel='linear', sigma_type='mpd' )
	# compare the HSIC value against numpy library
	#y = y.detach().cpu().numpy()
	#ŷ = ŷ.detach().cpu().numpy()
	#H2 = HSIC(ŷ, y, X_kernel='linear', Y_kernel='linear', sigma_type='mpd' )
	#import pdb; pdb.set_trace()
	return -H  

----------------------------------------------<br>
Create basic network and train

In [6]:
bNet = wuml.basicNetwork(costFunction, data, networkStructure=[(30,'relu'),('bn', True), (50,'relu'),('bn', True),(3,'none')], max_epoch=500, learning_rate=0.001)
bNet.train(print_status=True)
Ŷ = bNet(data, output_type='ndarray')		#Takes Numpy array or Tensor as input and outputs a Tensor

In [7]:
Network_out_v_Label = np.hstack((Ŷ, data.Y))
wuml.jupyter_print(Network_out_v_Label)

,0,1,2,3,4,5
0,1.215162,0.700804,-0.515642,1.0,0.0,0.0
1,1.297915,0.075018,-0.624129,1.0,0.0,0.0
2,1.375629,0.441589,-1.078524,1.0,0.0,0.0
3,1.776097,0.846963,-2.014745,1.0,0.0,0.0
4,0.624403,-0.085020,0.490469,1.0,0.0,0.0
...,...,...,...,...,...,...
173,0.253739,-0.113340,0.238930,0.0,0.0,1.0
174,0.353207,-0.049759,0.332164,0.0,0.0,1.0
175,0.923633,0.328500,0.250708,0.0,0.0,1.0
176,0.928336,0.334741,0.225715,0.0,0.0,1.0
